<a href="https://colab.research.google.com/github/basharE/EX1-Fizz-Buzz/blob/main/Fizzbuzz_primary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 5 : Primary Representation 

In [ ]:
import numpy as np
import sympy
import tensorflow.compat.v1 as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import sys
np.set_printoptions(threshold=sys.maxsize)
tf.disable_v2_behavior()

### Represent each input by an array of its binary digits.


In [ ]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

### Represent each input by an array of its primary represintation.


#### get all the primary numbers between two numers

In [ ]:
def get_primary(lower,upper):
  
  np_array = []

  for num in range(lower, upper + 1):
    if sympy.isprime(num) and num > 1:
      np_array.append(num)

  return np_array

####get the represintation of any primary number (with limited array of 10 elements, the 10th number represent counter of large primary numbers) 

In [ ]:
def get_primary_rep(upper):
  lower = 0
  np_array = get_primary(lower,upper)
  np_index = [0]*10
  i=0
  while upper > 1 :
    if (upper % np_array[i] == 0):
      if (i < 9):
        np_index[i] = np_index[i]+1
      else: np_index[9] = np_index[9]+1
      upper/= np_array[i]
      i=0
    else: i+=1
  return np_index

In [ ]:
def primary_encode():
  return np.array([get_primary_rep(i) for i in range(1,101)])

### One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]


In [ ]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

### Our goal is to produce fizzbuzz for the numbers 1 to 100. So it would be
### unfair to include these in our training data. Accordingly, the training data
### corresponds to the numbers 101 to (2 ** NUM_DIGITS - 1).



In [ ]:
NUM_DIGITS = 10
trX = np.array([get_primary_rep(i) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

### Our variables. The input has width NUM_DIGITS, and the output has width 4.


In [ ]:
X = tf.placeholder(float, [None, NUM_DIGITS])
Y = tf.placeholder(float, [None, 4])

### How many units in the hidden layer.


In [ ]:
NUM_HIDDEN = 2

### We'll want to randomly initialize weights.


In [ ]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

### Initialize the weights.


In [ ]:
w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

### Our model is a standard 1-hidden-layer multi-layer-perceptron with ReLU
### activation. The softmax (which turns arbitrary real-valued outputs into
### probabilities) gets applied in the cost function.

In [ ]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

### Predict y given x using the model.


In [ ]:
py_x = model(X, w_h, w_o)

### We'll train our model by minimizing a cost function.


In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=py_x))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

### And we'll make predictions by choosing the largest output.


In [ ]:
predict_op = tf.argmax(py_x, 1)

### Finally, we need a way to turn a prediction (and an original number)
### into a fizz buzz output

In [ ]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

#### Represent each input by an array of its binary digits.

In [ ]:
def fizz_buzz_encode_decimal(i):
    if   i % 15 == 0: return "fizzbuzz"
    elif i % 5  == 0: return "buzz"
    elif i % 3  == 0: return "fizz"
    else:             return i

####For every number will stay the same but if fizz it will be 0.3, buzz will be 0.5 and fizzbuzz will be 0.15

In [ ]:
def convert_fizzbuzz(scores):
  scores = np.where(scores=='fizzbuzz', 15, scores)
  scores = np.where(scores=='buzz', 5, scores)
  scores = np.where(scores=='fizz', 3, scores)
  return str(scores)

### Launch the graph in a session


In [ ]:
BATCH_SIZE = 128
# Launch the graph in a session
f= open("test_primary.txt","w+")
model_accuracy = [0]*10

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(10):
      for epoch in range(300):
            # Shuffle the data before each training iteration.
            p = np.random.permutation(range(len(trX)))
            trX, trY = trX[p], trY[p]

            # Train in batches of 128 inputs.
            for start in range(0, len(trX), BATCH_SIZE):
                end = start + BATCH_SIZE
                sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

            # And print the current accuracy on the training data.
            f.write(str(epoch)+ "  " + str(np.mean(np.argmax(trY, axis=1) == sess.run(predict_op, feed_dict={X: trX, Y: trY}))) + "\n")
            # print(epoch, np.mean(np.argmax(trY, axis=1) ==
            #                      sess.run(predict_op, feed_dict={X: trX, Y: trY})))

      # And now for some fizz buzz
      
      numbers = np.arange(1, 101)
      teX = primary_encode()

      teY = sess.run(predict_op, feed_dict={X: teX})
      classification_output = np.vectorize(fizz_buzz)(numbers, teY)

      f.write("\n########## classification_output ##########\n")
      f.write(str(classification_output))

      true_labels_output = np.array([fizz_buzz_encode_decimal(i) for i in range(1, 101)])
      f.write("\n########## true_labels_output ##########\n")
      f.write(str(true_labels_output))

      f.write("\n########## Classifier accuracy ##########\n")   
      accuracy = accuracy_score(true_labels_output,classification_output)
      model_accuracy[i] = accuracy
      f.write(str(accuracy))

      f.write("\n########## Confusion Matrix ##########\n")
      f.write(str(confusion_matrix(true_labels_output, classification_output)))
    
f.write("\n########## Model Accuracy 10x  ##########\n")

f.write(str(model_accuracy))

f.write("\n########## End  ##########\n")

f.close() 